In [25]:
import labels
import pandas as pd

from const import *

In [26]:
labels_df = labels.load_labels()

In [27]:
results = pd.read_csv("../results/crawler_results.csv")
results = results.rename(columns={"key": "filename", "value": "pred"})
results["pred"].unique()

array(['Review', 'False', 'True'], dtype=object)

In [28]:
def points(label, pred): 
    label_str, pred_str = str(label), str(pred)
    points = LOSS[label_str][pred_str]
    return points

def max_points(label): 
    label_str, pred_str = str(label), str(label)
    points = LOSS[label_str][pred_str]
    return points

In [29]:
merged = labels_df.merge(results, left_on="filename", right_on="filename")
merged["aligned"] = (merged["sensitive"] == (merged["pred"] == "True"))
merged["points"] = merged.apply(lambda row: points(row["sensitive"], row["pred"]), axis = 1)
merged["max_points"] = merged.apply(lambda row: max_points(row["sensitive"]), axis = 1)
merged

,filename,sensitive,name,extension,pred,aligned,points,max_points
0,within-resource.xml,True,within-resource,xml,True,True,20,20
1,fly-camera-book.xml,True,fly-camera-book,xml,True,True,20,20
2,illo-id-et-ullam.jpg,True,illo-id-et-ullam,jpg,True,True,20,20
3,voluptatem-minima.jpg,True,voluptatem-minima,jpg,True,True,20,20
4,debitis.msg,True,debitis,msg,True,True,20,20
...,...,...,...,...,...,...,...,...
843,escrow-ether.png,True,escrow-ether,png,True,True,20,20
844,export-t12-teneighty.jpg,True,export-t12-teneighty,jpg,True,True,20,20
845,huber-muster.png,True,huber-muster,png,True,True,20,20
846,meta-export.jpg,True,meta-export,jpg,True,True,20,20


In [30]:
merged[merged["extension"] == "pdf"]

,filename,sensitive,name,extension,pred,aligned,points,max_points
62,somebody-politics.pdf,True,somebody-politics,pdf,Review,False,10,20
63,fish-response-month.pdf,True,fish-response-month,pdf,Review,False,10,20
64,cultural-himself-a.pdf,True,cultural-himself-a,pdf,Review,False,10,20
65,keep-election-close.pdf,True,keep-election-close,pdf,Review,False,10,20
71,quasi-debitis.pdf,True,quasi-debitis,pdf,Review,False,10,20
...,...,...,...,...,...,...,...,...
682,store-fly-process.pdf,False,store-fly-process,pdf,True,False,-2,2
683,system-time-nation.pdf,False,system-time-nation,pdf,True,False,-2,2
687,develop-or-build.pdf,False,develop-or-build,pdf,Review,True,-1,2
703,everybody-place.pdf,False,everybody-place,pdf,Review,True,-1,2


In [31]:
perc_points = merged["points"].sum() / merged["max_points"].sum()
perc_points

0.6468492123030758

In [32]:
points_df = pd.DataFrame()
points_df["points"] = merged.groupby(by = "extension")["points"].sum()
points_df["max_points"] = merged.groupby(by = "extension")["max_points"].sum()
points_df

,points,max_points
extension,,
csv,396,396
db,112,112
docx,84,132
html,54,174
jpg,117,128
log,115,218
md,77,366
mp3,9,12
msg,50,50


In [33]:
print("Points:", merged["points"].sum(), "; Max points:", merged["max_points"].sum())

Points: 3449 ; Max points: 5332


In [34]:
merged[merged["extension"] == "log"]["aligned"].mean()
labels_df[labels_df["extension"] == "log"]["sensitive"].mean()

0.01

In [35]:
merged.groupby(by = "extension")["aligned"].mean().sort_values(ascending = False)

extension
csv      1.000000
mp3      1.000000
xml      1.000000
xlsx     1.000000
db       1.000000
msg      1.000000
pem      1.000000
log      0.990000
pub      0.975610
html     0.950000
py       0.900000
txt      0.900000
zip      0.900000
weird    0.872727
md       0.803030
jpg      0.800000
png      0.750000
docx     0.750000
ps1      0.666667
pdf      0.447619
Name: aligned, dtype: float64